<a href="https://colab.research.google.com/github/haa2wow/MEDICA/blob/main/34192_ZPI_CW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Połączenie z bazą danych SQLite (plik pacjenci.db będzie zapisany lokalnie)
conn = sqlite3.connect('pacjenci.db')
cursor = conn.cursor()

# Tworzymy tabelę pacjentów, jeżeli nie istnieje
cursor.execute('''
    CREATE TABLE IF NOT EXISTS pacjent (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        imie TEXT NOT NULL,
        nazwisko TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        haslo TEXT NOT NULL
    )
''')
conn.commit()


In [ ]:
# Tworzymy widżety dla formularza rejestracji
imie_input = widgets.Text(description='Imię:', placeholder="Podaj imię")
nazwisko_input = widgets.Text(description='Nazwisko:', placeholder="Podaj nazwisko")
email_input = widgets.Text(description='Email:', placeholder="podaj@example.com")
haslo_input = widgets.Password(description='Hasło:')

register_button = widgets.Button(description="Zarejestruj", button_style='success')
register_output = widgets.Output()

def on_register_clicked(b):
    with register_output:
        clear_output()
        imie = imie_input.value.strip()
        nazwisko = nazwisko_input.value.strip()
        email = email_input.value.strip()
        haslo = haslo_input.value.strip()

        if not (imie and nazwisko and email and haslo):
            print("Wszystkie pola muszą być wypełnione!")
            return

        # Wstawiamy dane do bazy, zwracamy błąd jeśli email jest już zarejestrowany
        try:
            cursor.execute('INSERT INTO pacjent (imie, nazwisko, email, haslo) VALUES (?, ?, ?, ?)',
                           (imie, nazwisko, email, haslo))
            conn.commit()
            print("Rejestracja przebiegła pomyślnie! Możesz teraz się zalogować.")
            # Czyszczenie pól formularza
            imie_input.value = ""
            nazwisko_input.value = ""
            email_input.value = ""
            haslo_input.value = ""
        except sqlite3.IntegrityError:
            print("Błąd: Email już istnieje w systemie.")

register_button.on_click(on_register_clicked)

# Wyświetlamy formularz rejestracji
registration_form = widgets.VBox([imie_input, nazwisko_input, email_input, haslo_input, register_button, register_output])
display(HTML("<h2>Formularz rejestracji pacjenta</h2>"))
display(registration_form)


In [ ]:
# Tworzymy widżety dla formularza logowania
login_email = widgets.Text(description='Email:', placeholder="podaj@example.com")
login_haslo = widgets.Password(description='Hasło:')
login_button = widgets.Button(description="Zaloguj", button_style='info')
login_output = widgets.Output()

def on_login_clicked(b):
    with login_output:
        clear_output()
        email = login_email.value.strip()
        haslo = login_haslo.value.strip()

        if not (email and haslo):
            print("Podaj email oraz hasło!")
            return

        cursor.execute('SELECT id, imie, nazwisko, email FROM pacjent WHERE email = ? AND haslo = ?', (email, haslo))
        result = cursor.fetchone()
        if result:
            patient_data = {
                'id': result[0],
                'imie': result[1],
                'nazwisko': result[2],
                'email': result[3]
            }
            show_dashboard(patient_data)
        else:
            print("Błędne dane logowania, spróbuj ponownie.")

login_button.on_click(on_login_clicked)

# Wyświetlamy formularz logowania
login_form = widgets.VBox([login_email, login_haslo, login_button, login_output])
display(HTML("<h2>Formularz logowania pacjenta</h2>"))
display(login_form)


In [ ]:
def show_dashboard(patient_data):
    clear_output()  # Czyścimy cały output notebooka – możesz też rozważyć inne rozwiązanie
    html_content = f"""
    <h2>Witaj, {patient_data['imie']} {patient_data['nazwisko']}!</h2>
    <p>Email: {patient_data['email']}</p>
    <p>ID pacjenta: {patient_data['id']}</p>
    """
    display(HTML(html_content))

def logout():
    clear_output()
    # Po wylogowaniu ponownie wyświetlamy formularze rejestracji i logowania
    display(HTML("<h2>Formularz rejestracji pacjenta</h2>"))
    display(registration_form)
    display(HTML("<h2>Formularz logowania pacjenta</h2>"))
    display(login_form)

# Umożliwiamy wylogowanie (funkcja logout dostępna globalnie)
globals()['logout'] = logout
